In [69]:
import os.path
import datetime
import nltk
import itertools

import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint
from collections import OrderedDict, Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

Use the index from *Compilers: Principles, Techniques, and Tools* to compare with our automatically generated index.

In [117]:
normalize_line = lambda line: line.strip().lower().replace('-', ' ')

with open('dragonIndex') as f:
    dragon_index = set([normalize_line(line) for line in f])

Read and parse .srt closed-caption files. Break them into 1 minute "documents"

In [5]:
FMT = '%H:%M:%S,%f'

def srt_to_strs(f, interval_length):
    '''Convert a .srt file to a dict of strings'''
    
    # [(text, start_time, end_time)]
    intervals = []
    while True:
        _seq_no = f.readline().strip()
        if _seq_no == '': break
        start_str, end_str = f.readline().strip().split(' --> ')
        start_time = datetime.datetime.strptime(start_str, FMT)
        end_time = datetime.datetime.strptime(end_str, FMT)
        
        text_lines = []
        while True:
            text_line = f.readline().strip()
            if text_line == '': break
            text_line = text_line.replace('&#39;', '')
            text_line = text_line.replace('&gt;', '')
            text_lines.append(text_line)
            
        text = ' '.join(text_lines)
        intervals.append((text, start_time, end_time))
        
    _text, interval_start_time, _end_time = intervals[0]
    
    result = OrderedDict()
    lecture_name = os.path.basename(f.name)[:-4]
    interval_lines = []
    for idx, (text, start_time, end_time) in enumerate(intervals):
        interval_lines.append(text)
        
        if idx == len(intervals) - 1 or end_time - interval_start_time > interval_length:
            result[(lecture_name, interval_start_time, end_time)] = ' '.join(interval_lines)
            interval_start_time = end_time
            interval_lines = []
            
    return result

In [6]:
SRT_FILE_NAMES = [
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/01-01-introduction-redo-correction.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/01-02-structure-of-a-compiler-final.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/01-03-economy-of-Programming-Languages_19m51s_.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/02-01-cool-overview-final.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/02-02-cool-example-ii-final.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/02-03-cool-example-iii-final-correction.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/03-01-Lexical-Analysis-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/03-02-lexical-analysis-examples-final.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/03-03-A+Regular+Languages.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/03-04-formal-languages.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/03-05-lexical-specifications-final-quizupdate.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/04+02+finite+automata+part+1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/04-01-lexical-specification.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/04-03-regular-expressions-to-nfas-final-quizupdate-correction.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/04-04-nfa-to-dfa-quizupdate.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/04-05-implementing-finite-automata-correction.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/05-01-introduction-to-parsing.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/05-02-A+Context+Free+Grammars.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/05-03-Derivations-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/05-04-A+Ambiguity.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-01-error-handling.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-02-abstract-syntax-trees.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-03-recursive-descent-parsing.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-04-1-recursive-descent-limitations-04-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-04-recursive-descent-algorithm.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/06-05-A+Left+Recursion.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-01-Predictive-Parsing-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-02-first-sets.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-03-follow-sets.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-04-ll1-parsing-tables.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-05-Bottom-Up-Parsing-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/07-06-Shift-Reduce-Parsing-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-01-Handles-Part-1.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-02-recognizing-handles.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-03-recognizing-viable-prefixes.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-04-valid-items.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-05-slr-parsing.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-06-slr-parsing-example.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-07-slr-improvements.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/08-08-slr-examples-correction.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-01-introduction-to-semantic-analysis.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-02-scope.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-03-symbol-tables.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-04-types.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-05-A+Type+Checking.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-06-A+Type+Environments.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-07-A+Subtyping.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-08-A+Typing+Methods.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/09-09-implementing-type-checking.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-01-A+Static+vs.+Dynamic+Typing.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-02-self-type.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-03-A+Self+Type+Operations.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-04-self-type-usage.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-05-A+Self+Type+Checking.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/10-06-error-recovery.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-01-runtime-organization.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-02-A+Activations.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-03-activation-records.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-04-globals-and-heap.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-05-alignment.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/11-06-stack-machines.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-01-introduction-to-code-generation.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-02-A+Code+Generation+I.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-03-A+Code+Generation+II.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-04-code-generation-example.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-05-A+Temporaries.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/12-06-A+Object+Layout.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/13-01-semantics-overview.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/13-02-operational-semantics.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/13-03-cool-semantics-i.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/13-04-A+Cool+Semantics+II.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/14-01-intermediate-code.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/14-02-optimization-overview.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/14-03-local-optimization.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/14-04-peephole-optimization.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/15-02-constant-propagation.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/15-03-analysis-of-loops.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/15-04-orderings.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/15-05-A+Liveness+Analysis.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/16-01-register-allocation.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/16-02-A+Graph+Coloring.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/16-03-A+Spilling.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/16-04-managing-caches.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/17-01-automatic-memory-management.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/17-02-A+Mark+and+Sweep.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/17-03-A+Stop+and+Copy.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/17-04-conservative-collection.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/17-05-A+Reference+Counting.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-01-java.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-02-java-arrays.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-03-java-exceptions.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-04-java-interfaces.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-05-java-coercions.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-06-java-threads.srt",
    "/Users/andrewlamb/Google_Drive/Stanford/CS199/CompilersSelfPacedCS1/18-07-other-topics.srt",
]

documents = OrderedDict()
for name in SRT_FILE_NAMES:
    with open(name) as f:
        documents.update(srt_to_strs(f, datetime.timedelta(minutes=1)))

POS tag the closed caption files, and store the words that occur for each part of speech.

In [ ]:
raw = ' '.join(documents.values())
sents = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(raw)]
tags = []
for sent in sents:
    try:
        tags.append(nltk.pos_tag(sent))
    except UnicodeDecodeError:
        pass

pos_to_tokens = defaultdict(set)
for sent in tags:
    for token, pos in sent:
        pos_to_tokens[pos].add(token.lower())

Lemmatize the closed caption files and remove stop words.

In [122]:
# http://www.ranks.nl/stopwords
with open('ranks_nl_stop_words_long.txt') as f:
    stop_words = [line.strip().replace("'", '') for line in f]
# A few added stop words
stop_words.extend([
        'thing',
        'things',
        'hello', 
        'going', 
        'uh', 
        'gonna', 
        'jack', 
        'will', 
        'alright',
        'cuz',
        'a0',
        'a1',
        'e0',
        'e1',
        'e2',
        'forgot',
        'graduate',
        'hope',
        'r1',
        's1',
        's2',
        't0',
        't1',
        't2',
        'x1',
        'a2i',
        'en',
        'bs',
        'idea',
        'keep',
        'sa',
        'gon',
        'eventually',
        'excuse'
    ])
# A few removed stop words
stop_words.remove('first')

lemmatizer = nltk.WordNetLemmatizer()
lemmatized_docs = OrderedDict()
for key, doc in documents.items():
    tokens = nltk.word_tokenize(doc)
    lemmatized_tokens = []
    for token in tokens:
        if token not in stop_words:
            try:
                lemmatized_tokens.append(lemmatizer.lemmatize(token))
            except:
                pass
            
    lemmatized_docs[key] = ' '.join(lemmatized_tokens)

pprint(documents.values()[0][:100])   
pprint(lemmatized_docs.values()[0][:100])

'Welcome to this course on compilers. My name is Alex Aiken. Im a professor here at Stanford Universi'
u'Welcome course compiler . My Alex Aiken . Im professor Stanford University . And talking implementat'


Run TFIDF. Grid search over parameters with Jaccard distance as a score.

In [137]:
grid_params = OrderedDict({
    'max_features': range(800, 2000, 100),
    'max_df': range(50, 100, 10)
})

value_combinations = itertools.product(*grid_params.values())
keyword_combinations = [{k: v for k, v in zip(grid_params.keys(), value_combination)} for value_combination in value_combinations]

scoring_fn = lambda auto_index: float(len(auto_index.intersection(dragon_index))) /  len(auto_index.union(dragon_index))

VALID_TAGS = [
    'NNP',
    'NNPS',
]

valid_tokens = set()
for pos in VALID_TAGS:
    valid_tokens = valid_tokens.union(pos_to_tokens[pos])
    
# Filter out tokens like 'abstract abstract'
no_repeats = lambda string: Counter(string.split()).most_common()[0][1] == 1

# Don't keep tokens like 'abstract' or 'abstract syntax tree' if there is another token 'abstract syntax tree'
unique_substring = lambda string: not any([string in other for other in vocabuluary - set([string])])

# Take strings that have any valid token.
has_valid = lambda string: any([token in valid_tokens for token in string.split()]) 

scores = {}
for keywords in keyword_combinations:
    tfidf = TfidfVectorizer(
        stop_words=stop_words,
        ngram_range=(1,4),
        **keywords
    )
    X = tfidf.fit_transform(lemmatized_docs.values())
    
    vocabuluary = set(tfidf.vocabulary_)
 
    auto_index = set(filter(
        lambda string: no_repeats(string) and unique_substring(string) and has_valid(string), 
        vocabuluary
    ))
    
    score = scoring_fn(auto_index)
    
    print '{}: {}'.format(keywords, score)
    
    scores[tuple(keywords.values())] = (score, auto_index)

max_score, best_index = max([tup for tup in scores.values()], key=lambda (score, index): score)
best_keywords = scores.keys()[np.argmax([score for score, index in scores.values()])]
print max_score
print best_keywords
pprint(best_index)

{'max_features': 800, 'max_df': 50}: 0.0275229357798
{'max_features': 800, 'max_df': 60}: 0.0277207392197
{'max_features': 800, 'max_df': 70}: 0.0288659793814
{'max_features': 800, 'max_df': 80}: 0.0288957688338
{'max_features': 800, 'max_df': 90}: 0.0279214064116
{'max_features': 900, 'max_df': 50}: 0.0299401197605
{'max_features': 900, 'max_df': 60}: 0.0311557788945
{'max_features': 900, 'max_df': 70}: 0.0301204819277
{'max_features': 900, 'max_df': 80}: 0.0301810865191
{'max_features': 900, 'max_df': 90}: 0.0293225480283
{'max_features': 1000, 'max_df': 50}: 0.030303030303
{'max_features': 1000, 'max_df': 60}: 0.0303921568627
{'max_features': 1000, 'max_df': 70}: 0.0324803149606
{'max_features': 1000, 'max_df': 80}: 0.0326086956522
{'max_features': 1000, 'max_df': 90}: 0.0317145688801
{'max_features': 1100, 'max_df': 50}: 0.0275142314991
{'max_features': 1100, 'max_df': 60}: 0.0287081339713
{'max_features': 1100, 'max_df': 70}: 0.031669865643
{'max_features': 1100, 'max_df': 80}: 0.

In [139]:
intersection = set(token.replace('-', ' ') for token in dragon_index).intersection(best_index)
print len(intersection)
pprint(intersection)

33
set([u'abstract syntax tree',
     u'assembly language',
     u'basic block',
     u'code generation',
     u'compile time',
     u'constant propagation',
     u'context free grammar',
     u'deterministic finite automaton',
     u'epsilon production',
     u'field',
     u'fortran',
     u'function call',
     u'garbage collection',
     u'intermediate code',
     u'java',
     u'lexical analyzer',
     u'literal',
     u'method call',
     u'optimization',
     u'parse tree',
     u'programming language',
     u'recursive descent',
     u'register allocation',
     u'regular expression',
     u'semantic analysis',
     u'shift reduce conflict',
     u'stack pointer',
     u'start symbol',
     u'syntax error',
     u'type checking',
     u'type expression',
     u'type variable',
     u'white space'])


In [140]:
false_negatives = dragon_index - best_index
print len(false_negatives)
pprint(false_negatives)

795
set(['acceptance',
     'accepting state',
     'access link',
     'action',
     'activation record',
     'activation tree',
     'actual parameter',
     'acyclic call string',
     'acyclic path',
     'acyclic test',
     'ada',
     'address',
     'address descriptor',
     'address space',
     'advancing edge',
     'affine array access',
     'affine expression',
     'affine partitioning',
     'affine space partition',
     'affine transformation',
     'aho corasick algorithm',
     'algebraic identities',
     'alias',
     'alignment',
     'allocation, of memory',
     'alpha',
     'alphabet',
     'ambiguous grammar',
     "amdahl's law",
     'analysis',
     'ancestor',
     'annotated parse tree',
     'anticipated expression',
     'antidependence',
     'antisymmetry',
     'antlr',
     'architecture',
     'arithmetic expression',
     'array',
     'array contraction',
     'ascii',
     'assembler',
     'associativity',
     'atom',
     'attribute',
  

In [141]:
possible_false_negatives = set([word for word in false_negatives if word in raw])
print len(possible_false_negatives)
pprint(possible_false_negatives)

263
set(['acceptance',
     'accepting state',
     'action',
     'activation record',
     'activation tree',
     'actual parameter',
     'ada',
     'address',
     'alias',
     'alignment',
     'alpha',
     'alphabet',
     'ambiguous grammar',
     'analysis',
     'ancestor',
     'architecture',
     'arithmetic expression',
     'array',
     'atom',
     'attribute',
     'augmented grammar',
     'automaton',
     'back edge',
     'back end',
     'block',
     'body',
     'bottom up parser',
     'branch',
     'bus',
     'bytecode',
     'c',
     'cache',
     'call',
     'calling sequence',
     'child',
     'chunk',
     'class',
     'clock',
     'closure',
     'coercion',
     'coloring',
     'comment',
     'communication',
     'concatenation',
     'conditional jump',
     'configuration',
     'conflict',
     'constant',
     'constant folding',
     'constraint',
     'control flow',
     'control link',
     'copy propagation',
     'cup',
     'dan

In [142]:
false_positives = best_index - dragon_index
print len(false_positives)
pprint(false_positives)

184
set([u'allocate object',
     u'allocation pointer',
     u'assembly code',
     u'attribute class',
     u'automata',
     u'automatic',
     u'beta',
     u'boolean',
     u'bottom parsing',
     u'call function',
     u'choose',
     u'class attribute',
     u'class definition',
     u'class implement',
     u'class method',
     u'class object',
     u'class type',
     u'clearly',
     u'code first',
     u'code program',
     u'computer',
     u'concrete',
     u'control flow graph',
     u'cool program',
     u'copy',
     u'count object',
     u'data structure',
     u'declared type',
     u'depending',
     u'deterministic automaton',
     u'dfa state',
     u'dollar',
     u'dynamic type',
     u'english',
     u'entry point',
     u'epsilon closure',
     u'epsilon first',
     u'epsilon move',
     u'executing',
     u'expression example',
     u'expression type',
     u'find',
     u'first argument',
     u'first evaluate',
     u'first position',
     u'first producti

In [157]:
with open('auto_index.txt', 'w') as f:
    for string in sorted(best_index):
        f.write(string + '\n')
        
with open('intersection.txt', 'w') as f:
    for string in sorted(intersection):
        f.write(string + '\n')
        
with open('false_negatives.txt', 'w') as f:
    for string in sorted(possible_false_negatives):
        f.write(string + '\n')

Lookup the minutes where "abstract syntax tree" and "recursive descent" appear.

In [55]:
token_to_docs = OrderedDict()
for key, column in filtered_vocabulary.items():
    document_indices = [idx for idx, count in enumerate(X[:, column].toarray().flatten()) if count > 0]
    token_to_docs[key] = sorted(list(set([documents.keys()[idx] for idx in document_indices])))
pprint([
        '{}: {} --> {}'.format(
            lecture_name, start_time.strftime(FMT), end_time.strftime(FMT)
        )
        for lecture_name, start_time, end_time in token_to_docs['recursive descent']
    ])

['06-03-recursive-descent-parsing: 00:00:03,949000 --> 00:01:04,720000',
 '06-03-recursive-descent-parsing: 00:01:04,720000 --> 00:02:08,110000',
 '06-04-1-recursive-descent-limitations-04-1: 00:00:00,560000 --> 00:01:05,609000',
 '06-04-1-recursive-descent-limitations-04-1: 00:04:09,370000 --> 00:05:13,000000',
 '06-04-1-recursive-descent-limitations-04-1: 00:05:13,000000 --> 00:06:17,830000',
 '06-04-recursive-descent-algorithm: 00:00:04,150000 --> 00:01:06,760000',
 '06-04-recursive-descent-algorithm: 00:08:32,039000 --> 00:09:36,120000',
 '06-04-recursive-descent-algorithm: 00:09:36,120000 --> 00:10:36,890000',
 '06-05-A+Left+Recursion: 00:00:03,570000 --> 00:01:07,810000',
 '06-05-A+Left+Recursion: 00:02:08,530000 --> 00:03:11,370000',
 '06-05-A+Left+Recursion: 00:04:12,380000 --> 00:05:14,970000',
 '07-01-Predictive-Parsing-Part-1: 00:00:03,780000 --> 00:01:04,059000',
 '07-01-Predictive-Parsing-Part-1: 00:02:06,979000 --> 00:03:12,010000',
 '07-05-Bottom-Up-Parsing-Part-1: 00:00

In [54]:
filtered_vocabulary = {k: v for k, v in tfidf.vocabulary_.items() if k in auto_index}
token_to_docs = OrderedDict()
for key, column in filtered_vocabulary.items():
    document_indices = [idx for idx, count in enumerate(X[:, column].toarray().flatten()) if count > 0]
    token_to_docs[key] = sorted(list(set([documents.keys()[idx] for idx in document_indices])))
pprint([
        '{}: {} --> {}'.format(
            lecture_name, start_time.strftime(FMT), end_time.strftime(FMT)
        )
        for lecture_name, start_time, end_time in token_to_docs['abstract syntax tree']
    ])

['06-02-abstract-syntax-trees: 00:00:04,799000 --> 00:01:08,590000',
 '06-02-abstract-syntax-trees: 00:02:13,450000 --> 00:03:13,690000',
 '06-02-abstract-syntax-trees: 00:03:13,690000 --> 00:03:48,260000',
 '09-03-symbol-tables: 00:00:03,530000 --> 00:01:05,950000',
 '09-03-symbol-tables: 00:01:05,950000 --> 00:02:06,859000',
 '09-03-symbol-tables: 00:02:06,859000 --> 00:03:07,750000',
 '09-03-symbol-tables: 00:03:07,750000 --> 00:04:10,299000',
 '09-03-symbol-tables: 00:04:10,299000 --> 00:05:14,180000',
 '09-04-types: 00:10:29,520000 --> 00:11:20,920000',
 '09-09-implementing-type-checking: 00:00:05,150000 --> 00:01:09,830000',
 '10-06-error-recovery: 00:01:06,900000 --> 00:02:08,060000',
 '10-06-error-recovery: 00:02:08,060000 --> 00:03:13,680000',
 '10-06-error-recovery: 00:03:13,680000 --> 00:04:15,739000',
 '10-06-error-recovery: 00:04:15,739000 --> 00:05:20,139000',
 '10-06-error-recovery: 00:05:20,139000 --> 00:06:25,000000',
 '12-02-A+Code+Generation+I: 00:12:28,710000 --> 00

In [102]:
filtered_vocabulary = {k: v for k, v in tfidf.vocabulary_.items() if k in best_index}
token_to_docs = OrderedDict()
for key, column in filtered_vocabulary.items():
    document_indices = [idx for idx, count in enumerate(X[:, column].toarray().flatten()) if count > 0]
    token_to_docs[key] = sorted(list(set([documents.keys()[idx] for idx in document_indices])))
pprint([
        '{}: {} --> {}'.format(
            lecture_name, start_time.strftime(FMT), end_time.strftime(FMT)
        )
        for lecture_name, start_time, end_time in token_to_docs['clearly']
    ])

['01-02-structure-of-a-compiler-final: 00:04:18,890000 --> 00:05:19,320000',
 '02-02-cool-example-ii-final: 00:13:38,889000 --> 00:14:43,829000',
 '03-04-formal-languages: 00:04:17,040000 --> 00:05:21,820000',
 '04-04-nfa-to-dfa-quizupdate: 00:03:12,739000 --> 00:04:17,609000',
 '06-01-error-handling: 00:01:05,990000 --> 00:02:08,820000',
 '06-01-error-handling: 00:03:11,780000 --> 00:04:15,610000',
 '06-03-recursive-descent-parsing: 00:03:08,420000 --> 00:04:11,209000',
 '06-04-1-recursive-descent-limitations-04-1: 00:03:05,750000 --> 00:04:09,370000',
 '07-02-first-sets: 00:11:39,610000 --> 00:12:39,850000',
 '07-02-first-sets: 00:12:39,850000 --> 00:13:43,740000',
 '07-03-follow-sets: 00:01:04,069000 --> 00:02:06,110000',
 '07-04-ll1-parsing-tables: 00:04:14,430000 --> 00:05:17,300000',
 '07-04-ll1-parsing-tables: 00:09:32,750000 --> 00:10:35,900000',
 '08-01-Handles-Part-1: 00:03:14,180000 --> 00:04:15,980000',
 '08-07-slr-improvements: 00:01:05,760000 --> 00:02:06,659000',
 '08-08